#Step 1 (Mandatory) - Install Txt to Video Finetuning

In [ ]:
#AI Lost Media's Text to Video Colab Workspace https://youtube.com/@ailostmedia
#huge thanks to Camenduru https://twitter.com/camenduru and Cerspense https://twitter.com/cerspense for putting these models together.
#tutorial: https://www.ailostmedia.com/post/the-ai-lost-media-text-to-video-colab-workspace
%cd /content
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install git+https://github.com/huggingface/diffusers transformers accelerate imageio[ffmpeg] -U einops omegaconf decord xformers==0.0.16 safetensors
!git clone -b dev https://github.com/camenduru/Text-To-Video-Finetuning
!git clone https://github.com/ailostmedia/Potat1ALM
!mv /content/Potat1ALM/inference.py /content/Text-To-Video-Finetuning/


#Step 2 (Mandatory) - Install Potat1 or ZeroScope (or both)

In [ ]:
#@title STEP2: Install Potat1
#default 1024 x 576 - try 800 x 448 for colab
%cd /content/
!git clone https://huggingface.co/camenduru/potat1

In [ ]:
#@title STEP2: Install ZeroScope 576
#default 576 x 320
%cd /content/
!git clone https://huggingface.co/cerspense/zeroscope_v2_576w

In [ ]:
#@title STEP2: Install ZeroScope XL
#default 1024 x 576 - try 800 x 448 for colab
%cd /content/
!git clone https://huggingface.co/cerspense/zeroscope_v2_XL

In [ ]:
#@title STEP2: Install ZeroScope 448
#default 448 x 256
%cd /content/
!git clone https://huggingface.co/cerspense/zeroscope_v2_dark_30x448x256

#Step 3 - Text to Video

In [ ]:
%cd /content/Text-To-Video-Finetuning
import torch
import random
import numpy as np
"""
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.cuda.manual_seed_all(2)
torch.manual_seed(0)
"""
#print("seed is " + str(torch.seed()))

#seeding = "Random"
#thisSeed = 123;


#preset = "Manual"
# while True:
#@markdown #### Be sure you have installed the model you want in step 2
model = "potat1" #@param ["potat1", "zeroscope_v2_dark_30x448x256", "zeroscope_v2_576w", "zeroscope_v2_XL"]
prompt = "extremely detailed, Futuristic Cityscape, blade runner, extremely cloudy, awardwinning, best quality, 8k" #@param {type:"string"}
negative = "text, watermark, copyright, blurry, nsfw, noise, quick motion, bad quality, flicker, dirty, ugly, fast motion, quick cuts, fast editing, cuts" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 25 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
width = 800 #@param {type:"raw"}
height = 448 #@param {type:"raw"}
fps = 10 #@param {type:"raw"}
num_frames = 30 #@param {type:"raw"}
seedManual = "Random"
seeding = "Random" #@param ["Random", "Manual"]

inputSeed = 7106521602475165645 #@param {type:"raw"}
if seeding == "Random":
  thisSeed = random.randint(0, ((1<<63)-1))
  print("seed is " + str(thisSeed))
else:
  thisSeed = inputSeed

thisHeight = int(round(height/8.0)*8.0)
thisWidth = int(round(width/8.0)*8.0)

thisModel="/content/"+model
!python inference.py -m {thisModel} -p {prompt} -n {negative} -W {thisWidth} -H {thisHeight} -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames} -seed {thisSeed}
#-seed {thisSeed}

#Optional: Image to Vid

In [ ]:
#@title Img2Vid Step 1: Install 3D Photo inpainting
!apt -y install -qq aria2 xvfb
!pip install vispy transforms3d networkx
%cd /content/
!git clone -b dev https://github.com/camenduru/3d-photo-inpainting
%cd /content/3d-photo-inpainting
!git clone https://github.com/camenduru/BoostingMonocularDepth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/3d-photo-inpainting/resolve/main/color-model.pth -d /content/3d-photo-inpainting/checkpoints -o color-model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/3d-photo-inpainting/resolve/main/depth-model.pth -d /content/3d-photo-inpainting/checkpoints -o depth-model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/3d-photo-inpainting/resolve/main/edge-model.pth -d /content/3d-photo-inpainting/checkpoints -o edge-model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/3d-photo-inpainting/resolve/main/model.pt -d /content/3d-photo-inpainting/checkpoints -o model.pt

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/3d-photo-inpainting/resolve/main/latest_net_G.pth -d /content/3d-photo-inpainting/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel -o latest_net_G.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/3d-photo-inpainting/resolve/main/model-f46da743.pt -d /content/3d-photo-inpainting/BoostingMonocularDepth/midas -o model.pt

The following additional packages will be installed:
  libaria2-0 libc-ares2 libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils
  xfonts-base xfonts-encodings xfonts-utils xserver-common
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2 libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils
  xfonts-base xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 12 newly installed, 0 to remove and 16 not upgraded.
Need to get 9,325 kB of archives.
After this operation, 17.4 MB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../00-libc-ares2_1.18.1-1ubuntu0.22.04.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../01-libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected pac

In [ ]:
#@title Img2Vid Step 2: Select Photo - Subject should be in center (use Jpg)
import os
import shutil
from google.colab import drive
from google.colab import files
def reformat_photo(photo):
    """function to reformat photo"""


#@markdown ### Select an uploading method
upload_method = "Upload" #@param ["Upload", "Custom Path"]


# remove previous input video
if os.path.isfile('/content/3d-photo-inpainting/image/test.jpg'):
    os.remove('/content/3d-photo-inpainting/image/test.jpg')

if upload_method == "Upload":
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, '/content/3d-photo-inpainting/image/test.jpg')
    PATH_TO_YOUR_PHOTO = '/content/3d-photo-inpainting/image/test.jpg'

elif upload_method == 'Custom Path':
    if not 'drive' in globals():
        drive.mount('/content/drive')
    #@markdown ``Add the full path to your video on your Gdrive `` 👇
    PATH_TO_YOUR_PHOTO = '/content/3d-photo-inpainting/image/test.jpg' #@param {type:"string"}
    if not os.path.isfile(PATH_TO_YOUR_PHOTO):
        print("ERROR: File not found!")
        raise SystemExit(0)

if upload_method == "Upload":
  print("Input photo")

else:
    if os.path.isfile(PATH_TO_YOUR_PHOTO):
        shutil.copyfile(PATH_TO_YOUR_PHOTO, "/content/3d-photo-inpainting/image/test.jpg")
        print("Input Photo")
        showVideo(PATH_TO_YOUR_PHOTO)

Saving normal statue.png to normal statue.png
Input photo


In [ ]:
#@title Img2Vid Step 3: Run 3D Photo inpainting
!pip install pyyaml
#@markdown ### Select the video type
video_type = "zoom-in" #@param ["zoom-in", "dolly-zoom-in", "circle", "swing"]
#@markdown ### Select the trajectory type (straight line to end video after move, circle to end at start)
trajectory_type = "double-straight-line" #@param ["double-straight-line", "circle"]
#@markdown ### Input the trajectories (use between 0 and .05)
#@markdown ### X (Negative is right, Positive is left)
x = 0.00 #@param {type:"raw"}
#@markdown ### Y (Negative is down, Positive is up)
y = -0.015 #@param {type:"raw"}
#@markdown ### Z (Negative is toward subject, Positive is away)
z = -0.05 #@param {type:"raw"}
#@markdown ### input the fps and frames for video
fps = 24 #@param {type:"raw"}
num_frames = 72 #@param {type:"raw"}

import yaml
%cd /content/3d-photo-inpainting

with open('argument.yml', 'r') as file:
  arguments = file.readlines()

#yaml_data[5] = "fps: "+ str(fps)
arguments[5] = "fps: "+ str(fps) +"\n"
arguments[6] = "num_frames: "+ str(num_frames) +"\n"
arguments[7] = "x_shift_range: [" +str(x) +"]\n"
arguments[8] = "y_shift_range: [" +str(y) +"]\n"
arguments[9] = "z_shift_range: [" +str(z) +"]\n"
arguments[10] = "traj_types: [" +str(trajectory_type) +"]\n"
arguments[11] = "video_postfix: [" +str(video_type) +"]\n"

with open('argument.yml', 'w') as file:
  file.writelines(arguments)

!xvfb-run -s "-screen 0 1280x720x24" python main.py --config argument.yml



/content/3d-photo-inpainting
/content/3d-photo-inpainting/mesh.py:965: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  passive_background = 10 if 10 is not None else background_thickness
/content/3d-photo-inpainting/mesh.py:966: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  passive_context = 1 if 1 is not None else context_thickness
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_o

In [ ]:
#@title Img2Vid STEP 4: V2V with ZeroScope (Will Need Interpolating)
%cd /content/Text-To-Video-Finetuning
import torch
import random
import numpy as np
"""
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.cuda.manual_seed_all(2)
torch.manual_seed(0)
"""
#print("seed is " + str(torch.seed()))

#seeding = "Random"
#thisSeed = 123;


#preset = "Manual"
# while True:
#@markdown ### Select the model (be sure to have installed in Step 2)
model = "zeroscope_v2_dark_30x448x256" #@param ["potat1", "zeroscope_v2_dark_30x448x256", "zeroscope_v2_576w", "zeroscope_v2_XL"]
#@markdown ### Copy Video Path from 3d-photo-inpainting/video
video_path = "" #@param {type:"string"}
video_weight = .2 #@param {type:"raw"}
#@markdown ### Fill in prompt and parameters!
prompt = "Extremely Detailed, dynamic shot, action, 80s live action medieval fantasy film shot of a young hero riding a hovercycle high in the clouds, stormy, hyperrealistic, best quality, awardwinning" #@param {type:"string"}
negative = "blurry, text, watermark, copyright, blurry, nsfw, noise, quick motion, bad quality, flicker, dirty, ugly, fast motion, quick cuts, fast editing, cuts" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 50 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
width = 448 #@param {type:"raw"}
height = 256 #@param {type:"raw"}
fps = 10 #@param {type:"raw"}
num_frames = 30 #@param {type:"raw"}
#@markdown ### Seeding not currently working for v2v (WIP)
seedManual = "Random"
seeding = "Random" #@param ["Random", "Manual"]

inputSeed = 6708511088475640657 #@param {type:"raw"}
if seeding == "Random":
  thisSeed = random.randint(0, ((1<<63)-1))
  print("seed is " + str(thisSeed))
else:
  thisSeed = inputSeed

thisHeight = int(round(height/8.0)*8.0)
thisWidth = int(round(width/8.0)*8.0)
thisModel="/content/"+model
thisVideoPath = str(video_path)
!python inference.py -m {thisModel} -p {prompt} -n {negative} -W {thisWidth} -H {thisHeight} -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames} -seed {thisSeed} -i {thisVideoPath} -iw {video_weight}
#-seed {thisSeed}

#Optional - V2V (Upload your own or use ZeroScope below)

In [ ]:
#@title V2V Step 1: Run ZeroScope (Optional)
%cd /content/Text-To-Video-Finetuning
import torch
import random
import numpy as np
"""
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.cuda.manual_seed_all(2)
torch.manual_seed(0)
"""
#print("seed is " + str(torch.seed()))

#seeding = "Random"
#thisSeed = 123;


#preset = "Manual"
# while True:

prompt = "extremely detailed, Futuristic Cityscape, blade runner, extremely cloudy, awardwinning, best quality, 8k" #@param {type:"string"}
negative = "text, watermark, copyright, blurry, nsfw, noise, quick motion, bad quality, flicker, dirty, ugly, fast motion, quick cuts, fast editing, cuts, blurry" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 25 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
fps = 10 #@param {type:"raw"}
num_frames = 30 #@param {type:"raw"}
seedManual = "Random"
seeding = "Random" #@param ["Random", "Manual"]

inputSeed = 5939699337684636079 #@param {type:"raw"}
if seeding == "Random":
  thisSeed = random.randint(0, ((1<<63)-1))
  print("seed is " + str(thisSeed))
else:
  thisSeed = inputSeed

!python inference.py -m "/content/zeroscope_v2_dark_30x448x256" -p {prompt} -n {negative} -W 448 -H 256 -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames} -seed {thisSeed}
#-seed {thisSeed}

In [ ]:
#@title V2V Step 2: Select Video (Upload from local drive or Gdrive)
import os
import shutil
from google.colab import drive
from google.colab import files
from IPython.display import HTML, clear_output
from base64 import b64encode
import moviepy.editor as mp


def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

def get_video_resolution(video_path):
    """Function to get the resolution of a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return (width, height)

def resize_video(video_path, new_resolution):
    """Function to resize a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    fourcc = int(video.get(cv2.CAP_PROP_FOURCC))
    fps = video.get(cv2.CAP_PROP_FPS)
    width, height = new_resolution
    output_path = os.path.splitext(video_path)[0] + '_720p.mp4'
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    while True:
        success, frame = video.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, new_resolution)
        writer.write(resized_frame)
    video.release()
    writer.release()

#@markdown ### Select an uploading method
upload_method = "Custom Path" #@param ["Upload", "Custom Path"]


# remove previous input video
if os.path.isfile('/content/sample_data/input_vid.mp4'):
    os.remove('/content/sample_data/input_vid.mp4')

if upload_method == "Upload":
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, '/content/sample_data/input_vid.mp4')
    PATH_TO_YOUR_VIDEO = '/content/sample_data/input_vid.mp4'

elif upload_method == 'Custom Path':
    if not 'drive' in globals():
        drive.mount('/content/drive')
    #@markdown ``Add the full path to your video on your Gdrive `` 👇
    PATH_TO_YOUR_VIDEO = '' #@param {type:"string"}
    if not os.path.isfile(PATH_TO_YOUR_VIDEO):
        print("ERROR: File not found!")
        raise SystemExit(0)

#@markdown <font color="orange">Notes:

#@markdown <font color="orange">. ``If your uploaded video is 1080p or higher resolution, this cell will resize it to 720p.``

#@markdown <font color="orange">. ``Do not upload videos longer than 60 seconds.``

#@markdown ___

video_duration = mp.VideoFileClip(PATH_TO_YOUR_VIDEO).duration
if video_duration > 60:
    print("WARNING: Video duration exceeds 60 seconds. Please upload a shorter video.")
    raise SystemExit(0)

video_resolution = get_video_resolution(PATH_TO_YOUR_VIDEO)
print(f"Video resolution: {video_resolution}")
if video_resolution[0] >= 1920 or video_resolution[1] >= 1080:
    print("Resizing video to 720p...")
    os.system(f"ffmpeg -i {PATH_TO_YOUR_VIDEO} -vf scale=1280:720 /content/sample_data/input_vid.mp4")
    PATH_TO_YOUR_VIDEO = "/content/sample_data/input_vid.mp4"
    print("Video resized to 720p")
else:
    print("No resizing needed")

if upload_method == "Upload":
  clear_output()
  print("Input Video")
  showVideo(PATH_TO_YOUR_VIDEO)
else:
    if os.path.isfile(PATH_TO_YOUR_VIDEO):
        shutil.copyfile(PATH_TO_YOUR_VIDEO, "/content/sample_data/input_vid.mp4")
        print("Input Video")
        showVideo(PATH_TO_YOUR_VIDEO)


In [ ]:
#@title V2V STEP 3: V2V with Potat1 (Will Need Interpolating)
%cd /content/Text-To-Video-Finetuning
import torch
import random
import numpy as np
"""
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.cuda.manual_seed_all(2)
torch.manual_seed(0)
"""
#print("seed is " + str(torch.seed()))

#seeding = "Random"
#thisSeed = 123;


#preset = "Manual"
# while True:
#@markdown ### Select the model (be sure to have installed in Step 2)
model = "potat1" #@param ["potat1", "zeroscope_v2_dark_30x448x256", "zeroscope_v2_567w", "zeroscope_v2_XL"]
video_weight = .3 #@param {type:"raw"}
prompt = "extremely detailed, Futuristic Cityscape, blade runner, extremely cloudy, awardwinning, best quality, 8k" #@param {type:"string"}
negative = "blurry, text, watermark, copyright, blurry, nsfw, noise, quick motion, bad quality, flicker, dirty, ugly, fast motion, quick cuts, fast editing, cuts" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 50 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
width = 600 #@param {type:"raw"}
height = 336 #@param {type:"raw"}
fps = 6 #@param {type:"raw"}
num_frames = 18 #@param {type:"raw"}
#@markdown ### Seeding not currently working for v2v (WIP)
seedManual = "Random"
seeding = "Random" #@param ["Random", "Manual"]

inputSeed = 6708511088475640657 #@param {type:"raw"}
if seeding == "Random":
  thisSeed = random.randint(0, ((1<<63)-1))
  print("seed is " + str(thisSeed))
else:
  thisSeed = inputSeed

thisHeight = int(round(height/8.0)*8.0)
thisWidth = int(round(width/8.0)*8.0)
thisModel="/content/"+model
!python inference.py -m {thisModel} -p {prompt} -n {negative} -W {thisWidth} -H {thisHeight} -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames} -seed {thisSeed} -i "/content/sample_data/input_vid.mp4" -iw {video_weight}
#-seed {thisSeed}